In [18]:
entity_list_file="D:\\Sola\\DEV\\SC\\NewsCrawler\\bert_modeller\\data\\entity_list.csv"
training_file="D:\\Sola\\DEV\\SC\\NewsCrawler\\bert_modeller\\data\\training_data_1_0_3000.csv"
model_files_output_folder="D:\\Sola\\DEV\\SC\\NewsCrawler\\bert_modeller\\models"
evaluation_file="D:\\Sola\\DEV\\SC\\NewsCrawler\\bert_modeller\\data\\evaluation_data.csv"
evaluation_output_file="D:\\Sola\\DEV\\SC\\NewsCrawler\\bert_modeller\\data\\evaluation_result.csv"

In [20]:
# import packages
import numpy as np
import pandas as pd

# optional text processing
from utils.text_processing import text_processing

# model training
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.metrics import classification_report
from scipy.special import softmax

import nltk
#nltk.download('wordnet')
#nltk.download('stopwords')

#Prepare file for training
df = pd.read_csv(training_file, header=0)
data = {}
df = pd.DataFrame(df[['text', 'label']])
df = df.rename({'label': 'labels'}, axis=1)


df['text'] = df.apply(lambda x: text_processing(x.text,                     
                lower=True, 
                remove_url=True, 
                remove_punctuation=True, 
                remove_stopwords=False, 
                replace_entity=True, 
                replace_hash=True,
                split_alphanumeric=False,
                lemmatize=True,
                stem=True), axis=1)

print("Check null: ", df.isnull().sum().sum())
print(df.head(3))

# initialise Model
model_args = ClassificationArgs(num_train_epochs=2, learning_rate = 5e-5, \
                                output_dir=model_files_output_folder,overwrite_output_dir= True
                                )
model = ClassificationModel(args = model_args, use_cuda = False,model_type = 'roberta', model_name = 'roberta-base')

# train the model
model.train_model(df)


Check null:  0
                                                text  labels
0            trezor wallet can be hack entity reveal       1
1  do anyon lose pac on entity i lose 40k on that...       1
2  happi to share with you our studi evalu risk o...       0


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 2 of 2: 100%|██████████| 2/2 [1:58:00<00:00, 3540.05s/it]


(1500, 0.44284875194417933)

In [21]:
#Prepare file for evaluation
df = pd.read_csv(evaluation_file, header=0)
data = {}
df = pd.DataFrame(df[['text', 'label']])
df = df.rename({'label': 'labels'}, axis=1)


df['text'] = df.apply(lambda x: text_processing(x.text,                     
                lower=True, 
                remove_url=True, 
                remove_punctuation=True, 
                remove_stopwords=False, 
                replace_entity=True, 
                replace_hash=True,
                split_alphanumeric=False,
                lemmatize=True,
                stem=True), axis=1)

print("Check null: ", df.isnull().sum().sum())
display(df.head(3))

Check null:  0


,text,labels
0,bitcoin be success today,0
1,the news be veri bad,1
2,hey there i m have problem disabl sim base otp...,1


In [22]:
#Evaluate the model
results = pd.DataFrame(columns = ['test_set', 'precision', 'recall', 'f1'])
# get predictions
pred, raw_outputs = model.predict(df['text'].tolist())

# append prediction (0 or 1) and probability (prob) to original dataframe
df['pred'] = pred
for i in range(len(df)):
    df.loc[i, 'raw_output_0'] = raw_outputs[i][0]
    df.loc[i, 'raw_output_1'] = raw_outputs[i][1]
probabilities = softmax(raw_outputs, axis=1)
probabilities_1 = [x[1] for x in probabilities]
df['prob'] = probabilities_1

df.to_csv(evaluation_output_file, index=False)
display(df.head(9))

100%|██████████| 2/2 [00:02<00:00,  1.01s/it]


,text,labels,pred,raw_output_0,raw_output_1,prob
0,bitcoin be success today,0,0,0.069261,-0.177180,0.438700
1,the news be veri bad,1,1,-0.821688,0.648291,0.813054
2,hey there i m have problem disabl sim base otp...,1,0,1.769145,-1.363101,0.041797
3,blockchain make me a milionair,0,1,-0.172034,-0.103994,0.517003
4,ciber thiev have be arrest,1,1,-2.050568,1.780158,0.978767
5,we make a fortun in bitcoin,0,0,1.319770,-1.022083,0.087716
6,blockchain a massiv opportun,0,1,-0.582014,0.447525,0.736826
7,not mention massiv entity print fraud,1,1,-2.080080,1.813349,0.980032
8,2 871 192 bitcoin long 14 785 5 07 11 20 21 48...,1,1,-0.424449,0.296838,0.672890
